<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/IsopropanolSynthesis_che4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook():
    from che4 import Range, Comp, CompArray, RangeArray, DotDict, d2nt, Props

In [4]:
import jax
import jax.numpy as jnp
from scipy.optimize import root

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [5]:
# Static parameters (Total feed, feed mole fradtions, feed temperature and )
# 1. propene
# 2. diisopropyl ether
# 3. isopropanol
# 4. water
# Rx1:  C3H6 + H2O -> C3H8O
# Rx2:  2C3H8O -> C6H14 + H2O 

#Simple version - Overall mass balance only

e = DotDict()
e.alkeneF = 100.
e.waterF = 100.

e.alkeney = jnp.array([1., 0., 0., 0.])
e.waterx = jnp.array([0., 0., 0., 1.])

e.rx1 = jnp.array([-1, 0, 1, -1])
e.rx2 = jnp.array([0, 1, -2, 1])

e.rx1_extent = Range(e.alkeneF/2, 0., e.alkeneF)
e.rx2_extent = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.alcoholP = Range(e.alkeneF/2, 0., e.alkeneF)  
e.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

e.etherP = Range(e.alkeneF/2, 0., e.alkeneF)
e.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

In [6]:
x0, x2nt, wrap, x2unk, const  = d2nt(e)

In [7]:
def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx 
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r

In [8]:
eqs_wrapped = wrap(eqs)
eqs_wrapped(x0)

DeviceArray([ 49.975, -24.25 , -47.9  ,  72.175], dtype=float64)

In [10]:
sol=root(eqs_wrapped, x0, jac=jax.jit(jax.jacobian(eqs_wrapped)))
print(sol)

    fjac: array([[-0.08077149, -0.07905158,  0.9898315 , -0.08637366],
       [-0.14383609,  0.70046632,  0.10451792,  0.69118313],
       [-0.88362986,  0.2265039 , -0.08891891, -0.39998459],
       [-0.4381614 , -0.67215606, -0.03745699,  0.59566584]])
     fun: array([ 2.45997710e-12,  8.81863471e-11, -1.42520662e-10,  2.28936869e-11])
 message: 'The solution converged.'
    nfev: 30
    njev: 2
     qtf: array([-1.56119367e-08,  1.05874020e-09,  1.17466932e-09, -2.60478164e-10])
       r: array([ 0.03809488, -9.42648571, -3.4174629 ,  0.22469471,  5.19258249,
       -0.51030967, -2.56314275,  0.41711272, -0.84829511,  2.46778622])
  status: 1
 success: True
       x: array([10.6251736 , -2.14547599,  2.97500876, -2.9744831 ])


In [11]:
nt=x2nt(sol.x)
nt._asdict()

OrderedDict([('alkeneF', DeviceArray(100., dtype=float64)),
             ('waterF', DeviceArray(100., dtype=float64)),
             ('alkeney', DeviceArray([1., 0., 0., 0.], dtype=float64)),
             ('waterx', DeviceArray([0., 0., 0., 1.], dtype=float64)),
             ('rx1', DeviceArray([-1,  0,  1, -1], dtype=int64)),
             ('rx2', DeviceArray([ 0,  1, -2,  1], dtype=int64)),
             ('rx1_extent', DeviceArray(99.9975704, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23773658, dtype=float64)),
             ('alcoholP', DeviceArray(95.14322505, dtype=float64)),
             ('alcoholPx',
              DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64)),
             ('etherP', DeviceArray(4.85920455, dtype=float64)),
             ('etherPx',
              DeviceArray([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64))])

In [12]:
unk=x2unk(sol.x)
unk._asdict()

OrderedDict([('rx1_extent', DeviceArray(99.9975704, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23773658, dtype=float64)),
             ('alcoholP', DeviceArray(95.14322505, dtype=float64)),
             ('etherP', DeviceArray(4.85920455, dtype=float64))])

In [13]:
const._asdict()

OrderedDict([('alkeneF', DeviceArray(100., dtype=float64)),
             ('waterF', DeviceArray(100., dtype=float64)),
             ('alkeney', DeviceArray([1., 0., 0., 0.], dtype=float64)),
             ('waterx', DeviceArray([0., 0., 0., 1.], dtype=float64)),
             ('rx1', DeviceArray([-1,  0,  1, -1], dtype=int64)),
             ('rx2', DeviceArray([ 0,  1, -2,  1], dtype=int64)),
             ('alcoholPx',
              DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64)),
             ('etherPx',
              DeviceArray([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64))])

In [14]:
_, r= eqs(x2nt(sol.x))
r

{'alkene': DeviceArray([100.,   0.,   0.,   0.], dtype=float64),
 'water': DeviceArray([  0.,   0.,   0., 100.], dtype=float64),
 'ether': DeviceArray([2.42960227e-03, 4.76202046e+00, 8.74656819e-02,
              7.28880682e-03], dtype=float64),
 'alcohol': DeviceArray([ 0.        ,  0.47571613, 89.43463155,  5.23287738], dtype=float64)}

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [15]:
# Include internal flows (recycle)


d=DotDict()
d.alkeneF = 100.
d.waterF = 100.

d.alkeney = jnp.array([1., 0., 0., 0.])
d.waterx = jnp.array([0., 0., 0., 1.])

d.rx1 = jnp.array([-1, 0, 1, -1])
d.rx2 = jnp.array([0, 1, -2, 1])

d.rx1_extent = Range(d.alkeneF/2, 0., d.alkeneF)
d.rx2_extent = Range(d.alkeneF/4, 0., d.alkeneF/2)

d.alcoholP = Range(d.alkeneF/2, 0., d.alkeneF)  
d.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

d.etherP = Range(d.alkeneF/2, 0., d.alkeneF)
d.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

d.WARFR = 12. #water_alkene_reactor_feed_ratio

d.D1F = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.D1Fx = Comp([0.001, 0.03, 0.07, 0.9])

d.D1K = jnp.array([100., 10., 5., 0.2])
d.D1Vy = Comp([0.01, 0.299, 0.3, 0.3])
d.D1Lx = Comp([0.0001, 0.1, 0.1, 0.8])
d.D1V = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)
d.D1L = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)

d.decantW = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.decantWx = jnp.array([0.0001, 0.0005, 0.0014, 0.98])


In [16]:
x0, x2nt, wrap, x2unk, const  = d2nt(d)

In [17]:
def eqs2(d):
    r=DotDict()
    r.reactor_feed = d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1L*d.D1Lx + d.decantW*d.decantWx

    r.REACTOR = r.reactor_feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - d.D1F*d.D1Fx

    r.FEED_RATIO = jnp.atleast_1d((d.waterF+d.D1L+d.decantW)/d.alkeneF - d.WARFR)
    
    r.D1 = d.D1F*d.D1Fx - d.D1V*d.D1Vy - d.D1L*d.D1Lx
    r.D1K = d.D1Vy/d.D1Lx - d.D1K

    r.AZ = d.D1V*d.D1Vy - d.etherP*d.etherPx - d.alcoholP*d.alcoholPx - d.decantW*d.decantWx

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return jnp.concatenate([r.REACTOR, r.FEED_RATIO, r.D1, r.D1K, r.AZ]), r


In [18]:
eqs2_wrapped = wrap(eqs2)

In [22]:
sol=root(eqs2_wrapped, x0, jac=jax.jit(jax.jacobian(eqs2_wrapped)))
sol.message

'The solution converged.'

In [23]:
unk=x2unk(sol.x)
unk._asdict()

OrderedDict([('rx1_extent', DeviceArray(99.9975704, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23773658, dtype=float64)),
             ('alcoholP', DeviceArray(95.14322505, dtype=float64)),
             ('etherP', DeviceArray(4.85920455, dtype=float64)),
             ('D1F', DeviceArray(1199.74805577, dtype=float64)),
             ('D1Fx',
              DeviceArray([3.50840994e-06, 8.53998248e-03, 2.16963767e-01,
                           7.74492742e-01], dtype=float64)),
             ('D1Vy',
              DeviceArray([3.37442255e-05, 4.60555454e-02, 7.86283218e-01,
                           1.67627493e-01], dtype=float64)),
             ('D1Lx',
              DeviceArray([3.37442255e-07, 4.60555454e-03, 1.57256644e-01,
                           8.38137465e-01], dtype=float64)),
             ('D1V', DeviceArray(113.87993521, dtype=float64)),
             ('D1L', DeviceArray(1085.86812057, dtype=float64)),
             ('decantW', DeviceArray(14.13187943, dtype=float

In [24]:
eqs2_wrapped(sol.x)

DeviceArray([ 5.98462252e-14,  7.10542736e-15, -1.13686838e-13,
             -1.13686838e-13,  0.00000000e+00,  4.55907014e-16,
             -7.99360578e-15,  2.84217094e-14,  2.27373675e-13,
             -7.10542736e-14, -7.10542736e-15,  8.88178420e-16,
             -1.38777878e-16, -8.65193334e-17,  1.93508404e-15,
              1.47451495e-14, -1.59872116e-14], dtype=float64)